In [7]:
import tensorflow as tf
import pandas as pd
import csv
import joblib
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import os
from tensorflow.keras.models import load_model

physical_devices = tf.config.list_physical_devices('GPU')

tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [8]:
# read a submitted csv file
data_dir = "/home/acapskyrim/projects/psm/psm_learn/data"
username = "user1"
model_name = username + "_model.h5"

for filename in os.listdir(data_dir):
    if username in filename:
        dataset = pd.read_csv(os.path.join(data_dir, filename), parse_dates=[0])
        dataset.columns = ['Time', 'Beacon_1', 'Beacon_2', 'Beacon_3', 'Beacon_4', 'Beacon_5']

dataset.set_index('Time', inplace=True)

In [9]:
# normalize data
scaler_name = "./" + username + "_scaler_data"
threshold_name = "./" + username + "_threshold"
scaler = joblib.load(scaler_name)
threshold = joblib.load(threshold_name)
x = scaler.transform(dataset)
x = x.reshape(x.shape[0], 1, x.shape[1])

In [10]:
data_out = {}

with tf.Graph().as_default():
    model = load_model(model_name)
    data_out["Analysis"] = []
#     model = define_model()
    preds = model.predict(x)
    preds = preds.reshape(preds.shape[0], preds.shape[2])
    preds = pd.DataFrame(preds, columns = dataset.columns)
    preds.index = dataset.index
    
    scored = pd.DataFrame(index=dataset.index)
    yhat = x.reshape(x.shape[0], x.shape[2])
    scored['Loss_mae'] = np.mean(np.abs(yhat - preds), axis=1)
    scored['Threshold'] = threshold
    scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
    
    triggered = []
    for i in range(len(scored)):
        temp = scored.iloc[i]
        if temp.iloc[2]:
            triggered.append(temp)
    print(len(triggered))
    if len(triggered) > 0:
        for j in range(len(triggered)):
            out = triggered[j]
            result = {"Anomaly": True, "date": f"{out.name.year:02d}" + "-" + f"{out.name.month:02d}" + "-" + f"{out.name.day:02d}", "time": f"{out.name.hour:02d}" + ":" + f"{out.name.minute:02d}"}
            data_out["Analysis"].append(result)

0


In [11]:
data_out

{'Analysis': []}

In [12]:
tf.keras.backend.clear_session()